In [ ]:
import glob
import os
from tqdm import tqdm
import numpy as np

import sys
sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

In [ ]:
sr = 16000
result_dir = "./audio/"
depth = 3
# speakers = [str(v).zfill(4) for v in np.arange(11, 21)]

models = {
    "ground-truth": "ground-truth/",
    # ground truth
    "External---gtHED": "External/gtHED/",
    "VA---gtHED": "Variance-Adaptor/gtHED/",
    "VA-seq---gtHED": "Variance-Adaptor_seq/gtHED/",
    # predicted HED
    "External---predHED_spkemb_noseq": "External/predHED---External---spkemb_noseq/",
    "External---predHED_spkemb": "External/predHED---External---spkemb/",
    "VA---predHED": "Variance-Adaptor/predHED---Variance-Adaptor/",
    "VA-seq---predHED": "Variance-Adaptor_seq/predHED---Variance-Adaptor_seq/",
}

groundtruth_dir = np.load("./audio/text_dir_LibriTTS.npy", allow_pickle=True).item()

# MUSHRA
## Speech Naturalness

In [ ]:
def init():
    whole_text = """var TestConfig = {
  "TestName": "Speech Naturalness Evaluation",
  "RateScalePng": "img/scale_abs.png",
  "RateScaleBgPng": "img/scale_abs_background.png",
  "RateMinValue": 0,
  "RateMaxValue": 100,
  "RateDefaultValue":50,
  "ShowFileIDs": false,
  "ShowResults": false,
  "LoopByDefault": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "RandomizeTestOrder": true,
  "MaxTestsPerRun": 20,
  "RequireMaxRating": false,
  "AudioRoot": "",
  "Testsets":
["""
    return whole_text

In [ ]:
def text_processing(text):
    text = text.replace('"', '`')
    ts = ["<br>"]
    for i, t in enumerate(text.split(" ")):
        ts += [t + " "]
        if i>0 and i%10==0:
            ts += ["<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"]
    text = "".join(ts)
    return text

In [ ]:
save = True
num = 100000
start = 0

whole_text = init()
modelnames = list(models.keys())
modelnames.remove("ground-truth")
filenames = glob.glob(result_dir + f"{models['ground-truth']}" + "*/"*depth + "*.wav")
filenames.sort()
for path in filenames[start:start+num]:
    basename = os.path.basename(path).split(".")[0]
    a = basename.split("_")
    d1, d2 = a[0], a[1]
    text = text_processing(groundtruth_dir[basename])
    
    starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "NAT---{basename}",
    "Text": "{text}",
    "Files": {{
      "ground_truth": "{path}",\n"""
    modelpaths = """"""
    for mn in modelnames:
        modelpath = result_dir + models[mn] + f"dev-clean/{d1}/{d2}/{basename}.wav"
        added = f'      "{mn}": "{modelpath}",\n'
        modelpaths += added
    modelpaths += f'    }}\n'
    modelpaths += f'  }},'
    whole_text += starter+modelpaths
whole_text += "\n]\n}}"
print(whole_text[:-1])
savefile = f"./config/mushra_speech_naturalness.js"
try:
    os.remove(savefile)
except FileNotFoundError:
    pass
if save:
    f = open(savefile, "a")
    f.write(whole_text[:-1])
    f.close()

## Speech Similarity

In [ ]:
def init():
    whole_text = """var TestConfig = {
  "TestName": "Speech Similarity Evaluation",
  "RateScalePng": "img/similar_scale.png",
  "RateScaleBgPng": "img/scale_abs_background.png",
  "RateMinValue": 0,
  "RateMaxValue": 100,
  "RateDefaultValue":50,
  "ShowFileIDs": false,
  "ShowResults": false,
  "LoopByDefault": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": true,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "RandomizeTestOrder": true,
  "MaxTestsPerRun": 20,
  "RequireMaxRating": false,
  "AudioRoot": "",
  "Testsets":
["""
    return whole_text

In [ ]:
save = True
num = 100000
start = 0

whole_text = init()
modelnames = list(models.keys())
modelnames.remove("ground-truth")
filenames = glob.glob(result_dir + f"{models['ground-truth']}" + "*/"*depth + "*.wav")
filenames.sort()
for path in filenames[start:start+num]:
    basename = os.path.basename(path).split(".")[0]
    a = basename.split("_")
    d1, d2 = a[0], a[1]
    
    starter = f"""
  {{
    "Name": "{basename}",
    "TestID": "SIM---{basename}",
    "Files": {{
      "Reference": "{path}",\n"""
    modelpaths = """"""
    for mn in modelnames:
        modelpath = result_dir + models[mn] + f"dev-clean/{d1}/{d2}/{basename}.wav"
        added = f'      "{mn}": "{modelpath}",\n'
        modelpaths += added
    modelpaths += f'    }}\n'
    modelpaths += f'  }},'
    whole_text += starter+modelpaths
whole_text += "\n]\n}}"
print(whole_text[:-1])
savefile = f"./config/mushra_speech_similarity.js"
try:
    os.remove(savefile)
except FileNotFoundError:
    pass
if save:
    f = open(savefile, "a")
    f.write(whole_text[:-1])
    f.close()